# Escalamiento de los datos

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

> Diversos algoritmos son sensibles a la escala en la que viene cada feature. **Re-escalarlos** puede traer significativas mejoras de rendimiento.


Existen distintas estrategias de escalamiento de tus features, pero **la más común es la estandarización** donde convertimos la variable para que la distribución de esta siga una distribución que es Gaussiana de media 0 y de desviación estandar 1.

In [0]:
from sklearn.model_selection import train_test_split

X = pd.read_csv('drive/My Drive/Colab Notebooks/datasets/machine_learning_python/intermediate_results/X.csv')
y = X['worldwide_gross']
X = X.drop('worldwide_gross', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [5]:
scaler.mean_

array([3.36892195e+07, 2.00217053e+03, 2.13230735e+00, 1.08280715e+02,
       1.03218418e+04, 3.79926408e+07, 6.46052632e+00, 4.90935746e+07])

In [6]:
# Desviación estándar de cada feature
scaler.scale_

array([4.12401191e+07, 1.19305015e+01, 7.61938031e-01, 2.24599838e+01,
       2.00395939e+04, 7.55028813e+07, 1.05964029e+00, 6.52379435e+07])

In [9]:
X.values

array([[4.25000000e+08, 2.00900000e+03, 1.78000000e+00, ...,
        2.37000000e+08, 7.90000000e+00, 7.60505847e+08],
       [3.06000000e+08, 2.00213073e+03, 2.12697615e+00, ...,
        4.04553863e+07, 7.10000000e+00, 4.83193329e+07],
       [3.00000000e+08, 2.00700000e+03, 2.35000000e+00, ...,
        3.00000000e+08, 7.10000000e+00, 3.09404152e+08],
       ...,
       [7.00000000e+03, 2.00500000e+03, 2.12697615e+00, ...,
        3.25000000e+03, 7.80000000e+00, 4.83193329e+07],
       [3.96700000e+03, 2.01200000e+03, 2.35000000e+00, ...,
        4.04553863e+07, 6.30000000e+00, 1.04430000e+04],
       [1.10000000e+03, 2.00400000e+03, 1.85000000e+00, ...,
        1.10000000e+03, 6.60000000e+00, 8.52220000e+04]])

In [7]:
# X transformado
scaler.transform(X_train)

array([[ 1.60791923e+00,  6.56256986e-01,  2.85709129e-01, ...,
         8.21258183e-01,  2.25995262e-01,  1.07493277e+00],
       [ 1.04528808e-01, -9.81119748e-02, -3.70512216e-01, ...,
         9.74685532e-05, -9.06464511e-01, -4.03487743e-01],
       [ 1.24419574e+00,  7.40075760e-01, -3.70512216e-01, ...,
         8.21258183e-01,  6.03481853e-01, -4.06677988e-02],
       ...,
       [ 2.74266436e-01, -5.17205842e-01,  2.85709129e-01, ...,
         9.28091622e-02, -4.34606273e-01, -4.44399563e-01],
       [-1.98574100e-01,  2.37163119e-01, -3.70512216e-01, ...,
        -3.04526668e-01,  9.80968444e-01, -2.38024051e-01],
       [ 1.12295458e+00, -1.42932014e-02,  2.85709129e-01, ...,
         5.56367630e-01,  5.09110205e-01,  3.69750089e-01]])

In [0]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [11]:
from sklearn.linear_model import Lasso

model = Lasso()
model_scaled = Lasso()

model.fit(X_train, y_train)
model_scaled.fit(X_train_scaled, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [12]:
print(model.score(X_test, y_test))
print(model_scaled.score(X_test_scaled, y_test))

0.7712814192844308
0.7712814223855115



> Los modelos de regresión no se ven afectados por el escalamiento de las features. Los de clasificación sí.

# Simplificar las transformaciones con pipelines

Para hacer tu código más reproducible, y para evitar tener que aplicar multiples veces una misma transformación te recomendamos utilizar  `sklearn.pipeline.make_pipeline`  que permite encadenar transformaciones a tus modelos.

In [13]:
from sklearn.pipeline import make_pipeline
model_scaled = make_pipeline(StandardScaler(),
                             Lasso())

model_scaled.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('lasso',
                 Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

In [14]:
print(model_scaled.score(X_test, y_test))

0.7712814223855115


# Crear nuevas features de forma automática

In [15]:
A = np.arange(6).reshape(3,2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [16]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
transformer.fit(A)
transformer.transform(A)

# transformer.fit_transform(A)  # => Hace las 2 acciones,entrena y transforma

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

`PolynomialFeatures` transforma una matriz $(A1,A2)$ a $(1,A1,A2,A1^2,A1\cdot A2,A2^2)$

In [17]:
X.shape

(4104, 8)

In [19]:
transformer = PolynomialFeatures(2)

# El transformador de ScikitLearn genera mas features a partir de la dimensión polinómica
transformer.fit_transform(X).shape

(4104, 45)

In [20]:
# Usando los features que crea ScikitLearn
model_poly = make_pipeline(PolynomialFeatures(2),
                           Lasso())

model_poly.fit(X_train, y_train)
model_poly.score(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.865903739246003e+18, tolerance: 9727882124860768.0
  positive)


0.7637791729096762

In [21]:
# Usando los datos normales
model = Lasso()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.7712814192844308

# Crear features categóricas

> En terminos de Machine Learning a las features que pueden tomar un número finito de valores se les llama categóricas. Ejemplos para esto són: género, páis, grado académico, etc.


Un mapeo del tipo $\{Perú, Chile, Colombia, Venezuela\} \rightarrow \{1, 2, 3, 4\}$ tiene el problema de asignarle un ordén a los valores posibles de la categoría. Este orden impacta de distintas maneras los algoritmos de Machine Learning, por ejemplo aquellos que dependen de la topología de $R^n$ y de la función de distancia entre puntos en este espacio, considerarán que ciertas categorías se encuentran más cercanas unas de otras, siendo que esto es generado puramente artificialmente por el encoding, y no por las datos per se.

Para no introducir información falsa o erronéa en nuestro modelos existen formas más inteligentes de encodear nuestros datos.

**Encoding one-hot**

Este encoding consiste en asignarle una columna a cada categoría y rellenarla con 0 y 1 de la forma siguiente:

In [22]:
d = pd.DataFrame([['Chile','Colombia','Colombia','Venezuela'],['hombre','mujer','hombre','mujer']])
d = d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [23]:
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


Sklearn también ofrece un objeto OneHotEncoder pero es un poco más díficil de utilizar, así que por criterios pedagogicos hemos elegido pd.get_dummies. Sin embargo el objeto de sklearn tiene la ventaja de ser pipeable, por lo que es bueno considerarlo para ciertos casos de uso.

Cuantas columnas generaríamos con un one-hot encoding de nuestras features categóricas?

In [0]:
movies_obj = pd.read_csv('drive/My Drive/Colab Notebooks/datasets/machine_learning_python/movies_obj.csv')

In [27]:
movies_obj.apply(pd.Series.nunique).sort_values()

color                2
content_rating      18
language            47
country             65
genres             914
actor_1_name      2097
director_name     2398
actor_2_name      3032
actor_3_name      3521
plot_keywords     4760
movie_title       4917
dtype: int64


Las features más informativas son las del casting. Si embargo haciendo un one-hot encoding de estas estaríamos aumentando la dimensión por 2000 y algo!!

# Encoding Binario

Esta técnica no es canónica por lo que tendremos que buscarla en otra librería. Sin embargo el autor tuvo la buena idea de hacer su API compatible con la de sklearn, así que no tendremos ninguna dificultad en usarla.

$$ Categoria \rightarrow Numero \rightarrow Binario \rightarrow Columnas $$

In [29]:
!pip install category_encoders

In [0]:
categoricals = pd.read_csv('drive/My Drive/Colab Notebooks/datasets/machine_learning_python/categoricals.csv').set_index('Unnamed: 0')

In [36]:
categoricals.head(1)

,actor_1_name,director_name
Unnamed: 0,,
0,CCH Pounder,James Cameron


In [0]:
categoricals = categoricals.reset_index(drop=True).fillna(0)

In [0]:
X_binenc = pd.concat([X, categoricals], axis=1)

In [64]:
X_binenc.head()

,production_budget,title_year,aspect_ratio,duration,cast_total_facebook_likes,budget,imdb_score,gross,actor_1_name,director_name
0,425000000.0,2009.000000,1.780000,178.000000,4834.0,2.370000e+08,7.9,7.605058e+08,CCH Pounder,James Cameron
1,306000000.0,2002.130733,2.126976,108.577186,143.0,4.045539e+07,7.1,4.831933e+07,Doug Walker,Doug Walker
2,300000000.0,2007.000000,2.350000,169.000000,48350.0,3.000000e+08,7.1,3.094042e+08,Johnny Depp,Gore Verbinski
3,300000000.0,2015.000000,2.350000,148.000000,11700.0,2.450000e+08,6.8,2.000742e+08,Christoph Waltz,Sam Mendes
4,275000000.0,2012.000000,2.350000,164.000000,106759.0,2.500000e+08,8.5,4.481306e+08,Tom Hardy,Christopher Nolan


In [0]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name','director_name'])

In [66]:
encoder.fit_transform(X_binenc).shape

(4104, 32)

In [0]:
X_binenc = encoder.fit_transform(X_binenc)

In [0]:
Xb_train, Xb_test, y_train, y_test = train_test_split(X_binenc, y)

In [0]:
model_binenc = Lasso()
model = Lasso()

In [70]:
model_binenc.fit(Xb_train,y_train)
model.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.117813713808982e+18, tolerance: 8763783009743717.0
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.6463411890329846e+18, tolerance: 8763783009743717.0
  positive)


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [71]:
print(model_binenc.score(Xb_test,y_test))
print(model.score(X_test,y_test))

0.7963666057248362
-0.02248399803996981



Aumentamos el rendimiento de nuestro algoritmo pero no de forma significativa. Mantengamos entonces la dimensionalidad de nuestro espacio de features baja, y vamos a buscar modelos más complejos

# Conocimiento experto

> Una grán parte del diseño de las features pasa por un **conocimiento espécifico del dominio en el que se esta trabajando**.
Por ejemplo para analizar una imagen nuestro cerebro no se concentra en los millones de pixeles de una imagen, pero sólo en algunos relevantes como los de los contornos. Durante un buen tiempo **los sistemas de visión de computadores encodeaban features que traducían este conocimiento experto (contornos).**
Una de las únicas formas de obtener este conocimiento de forma sistemática es ir a bucear en repositorios de papers de Machine Learning como Arxiv, y estudiar la investigación que se ha hecho sobre el dominio específico.

# Más datos de calidad



---



Nada le gana conseguir más datos que sean encodeables en features de calidad.

***Píramide de Maslow del Machine Learning***

![Maslow](https://github.com/JuanPabloMF/machine-learning-platzi/raw/8c90f12d2f94ba2b58d725d2eb232a8d654cfc22/img/maslow.png)

Contamos con la base de datos de ganancias de las péliculas el primer fin de semana de exhibición, así como la cantidad de cines en la que fue estrenada.

In [73]:
pd.read_csv('drive/My Drive/Colab Notebooks/datasets/machine_learning_python/opening_df.csv')

,Unnamed: 0,movie_title,opening_gross,screens
0,0,10 Days in a Madhouse,2451.0,10.0
1,1,10 Things I Hate About You,8330681.0,2271.0
2,2,102 Dalmatians,19883351.0,2704.0
3,3,12 Rounds,5329240.0,2331.0
4,4,12 Years a Slave,923715.0,19.0
...,...,...,...,...
2267,2271,Zombieland,24733155.0,3036.0
2268,2272,Zookeeper,20065617.0,3482.0
2269,2273,Zoolander 2,15650000.0,3394.0
2270,2274,Zoom,4510408.0,NaN


Puedes mejorar considerablemente nuestra predicción?